## VARIABLES DE CONFIGURACION

In [3]:
#PATH = 'DATA/tobii/'

USER_ID = 'ALEX'


In [9]:
global_csv_filename = USER_ID+"_gaze_data.csv"
event_csv_filename = USER_ID+"_events_data.csv"

In [16]:
import os

if os.path.exists(global_csv_filename):
    print("CAMBIAR NOMBRE USER_IR")
else:
    print("EXPERIMENTO PARA EL USUARIO: "+global_csv_filename)

EXPERIMENTO PARA EL USUARIO: DATA/tobii/ALEX_gaze_data.csv


## Importar librerias

In [17]:
import sys
print(sys.version)

3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]


In [18]:
import tobii_research as tr

In [19]:
import time
import csv
from collections import defaultdict

In [20]:
from pynput.keyboard import Key, Listener

Inicializo diccionario para contar los eventos (pulsar la tecla e)

In [21]:
global_user_position_guide = None

## Comprobar conexión con Tobii

In [22]:
found_eyetrackers = tr.find_all_eyetrackers()

In [23]:
my_eyetracker = found_eyetrackers[0]

In [24]:
print("Address: " + my_eyetracker.address)
print("Model: " + my_eyetracker.model)
print("Name: " + my_eyetracker.device_name)
print("Serial number: " + my_eyetracker.serial_number)

Address: tobii-prp://TPNA1-030251340594
Model: Tobii Pro Nano
Name: 
Serial number: TPNA1-030251340594


Función que saca las **posiciones X e Y de cada ojo** y lo escribe en un csv:

Iba a plantearla sobreescribiendo el archivo de datos cada vez que se analice, pero la he dejado así por si queremos tener diferentes archivos con datos particulares, simplemente cambiar el nombre del csv a crear y así podremos crear distintos csvs con distintos datos con la misma función.

**IMPORTANTE: BORRAR EL CSV ANTES DE HACER QUE ESTA FUNCIÓN SE EJECUTE**

In [25]:
def gaze_data_callback(gaze_data):
    
#    print("Left eye: ({gaze_left_eye}, left pupil: {pupil_left_eye}) \t Right eye: ({gaze_right_eye}, Right pupil: {pupil_right_eye}, time: {timestamp})".format(
#    gaze_left_eye=gaze_data['left_gaze_point_on_display_area'],
#    gaze_right_eye=gaze_data['right_gaze_point_on_display_area'],
#    pupil_left_eye=gaze_data['left_pupil_diameter'],
#    pupil_right_eye=gaze_data['right_pupil_diameter'],
#    timestamp=gaze_data['device_time_stamp']))
    
    global event_count, key_pressed, global_csv_filename, event_csv_filename
    
    # Guardar los datos en un archivo CSV con nombres de columnas
    csv_filename = global_csv_filename

    # Definir nombres de columnas
    fieldnames = ['Left Eye X', 'Left Eye Y', 'Right Eye X', 'Right Eye Y', 'Pupil left eye', 'Pupil right eye', 'Timestamp device', 'Timestamp system']
#    fieldnames = ['Left Eye X', 'Left Eye Y', 'Right Eye X', 'Right Eye Y', 'Pupil left eye', 'Pupil right eye', 'Timestamp']
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Si el archivo está vacío, escribir encabezados
        if file.tell() == 0:
            writer.writeheader()

        # Obtener datos relevantes y formatearlos adecuadamente
        left_eye_data = gaze_data['left_gaze_point_on_display_area']
        right_eye_data = gaze_data['right_gaze_point_on_display_area']
        left_eye_pupil = gaze_data['left_pupil_diameter']
        right_eye_pupil = gaze_data['right_pupil_diameter']
        timestamp_device = gaze_data['device_time_stamp']
        timestamp_system = time.time()

        # Asegurarse de que los datos son del tipo adecuado para la escritura CSV
        left_eye_data = tuple(map(float, left_eye_data))
        right_eye_data = tuple(map(float, right_eye_data))
        

        # Crear un diccionario con nombres de columnas
        row_dict = {
            'Left Eye X': left_eye_data[0],
            'Left Eye Y': left_eye_data[1],
            'Right Eye X': right_eye_data[0],
            'Right Eye Y': right_eye_data[1],
            'Pupil left eye': left_eye_pupil,
            'Pupil right eye': right_eye_pupil,
            'Timestamp device': timestamp_device,
            'Timestamp system': timestamp_system
        }

        # Escribir datos en el archivo CSV
        writer.writerow(row_dict)

In [26]:

with open(event_csv_filename, mode='a', newline='') as file:
    event_writer = csv.writer(file)
    if file.tell() == 0:
        event_writer.writerow(['Evento','Timestamp'])

def on_press(key):
    print('{0} pressed'.format(key))

def on_release(key):
    timestamp = time.time()
    print('{0} release'.format(key))
    with open(event_csv_filename, mode='a', newline='') as file:
        event_writer = csv.writer(file)
        event_writer.writerow([f'Evento {key}', timestamp])

    if key == Key.esc:
        # Stop listener
        return False

# Collect events until released
listener = Listener(on_press=on_press, on_release=on_release)


## EMPEZAR GRABACION

In [27]:
if os.path.exists(global_csv_filename):
    print("CAMBIAR NOMBRE USER_IR")
else:
    print("COMIENZO EXPERIMENTO PARA EL USUARIO: "+global_csv_filename)
    my_eyetracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, gaze_data_callback, as_dictionary=True)
    listener.start()

COMIENZO EXPERIMENTO PARA EL USUARIO: DATA/tobii/ALEX_gaze_data.csv
'1' pressed
'1' release
'2' pressed
'2' release
'2' pressed
'2' release
'3' pressed
'3' release
'c' pressed
'c' release
'4' pressed
'4' release
'5' pressed
'5' release
'6' pressed
'6' release


##### PARAR GRABACION

In [28]:
my_eyetracker.unsubscribe_from(tr.EYETRACKER_GAZE_DATA, gaze_data_callback)
listener.stop()

In [15]:
#https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html